# Data Science 5 : TP 5a - Traitement d'Images avec Python et Filtres Morphologiques

Enseignant : Jean Delpech

Cours : Data Science

Classe : M1 Data/IA

Année scolaire : 2025/2026

Dernière mise à jour : janvier 2026

## Module Data Science M1 - Séances 11 & 12

**Objectifs du TP :**

Ce TP fait suite au cours théorique sur le traitement d'images. Vous allez maintenant utiliser les bibliothèques **OpenCV** et **scikit-image** pour appliquer les concepts vus précédemment :

- Seuillage et binarisation
- Convolution et filtrage
- Morphologie mathématique

À l'issue de ce TP, vous saurez :
- Choisir la bibliothèque appropriée selon le cas d'usage
- Appliquer efficacement les opérations de traitement d'images
- Comparer les performances des implémentations

## 1. Installation et imports

### 1.1 Installation (si nécessaire)

```bash
pip install opencv-python scikit-image numpy matplotlib
```

In [ ]:
!pip install opencv-python scikit-image numpy matplotlib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import skimage
from skimage import io, filters, morphology, exposure, color, util
from skimage.filters import threshold_otsu, gaussian, sobel
from skimage.morphology import disk, square, erosion, dilation, opening, closing
import time

# Vérification des versions
print(f"OpenCV version : {cv2.__version__}")
print(f"scikit-image version : {skimage.__version__}")
print(f"NumPy version : {np.__version__}")

### 1.2 Fonctions utiles

Pendant le cours vous aurez remarqué qu’on affiche beaucoup d’image. Créons donc une fonction pour afficher des images (vous pourrez sinon utiliser la fonction, utilisée en cours qui affiche une image et son histogramme, si vous voulez accéder à plus d’information).

In [ ]:
def afficher_images(images, titres, cmap='gray', figsize=(15, 5)):
    """
    Affiche plusieurs images côte à côte.
    
    Paramètres
    ----------
    images : list
        Liste des images à afficher
    titres : list
        Liste des titres correspondants
    cmap : str
        Colormap à utiliser
    figsize : tuple
        Taille de la figure
    """
    n = len(images)
    fig, axes = plt.subplots(1, n, figsize=figsize)
    if n == 1:
        axes = [axes]
    
    for ax, img, titre in zip(axes, images, titres):
        if len(img.shape) == 3 and img.shape[2] == 3:
            ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        else:
            ax.imshow(img, cmap=cmap)
        ax.set_title(titre, fontweight='bold')
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()

Dans ce TP nous allons nous familiariser et comparer deux bibliothèques : OpenCV et Scikit-Images. Il peut être intéressant d’évaluer le temps d’exécution pour voir laquelle est la plus rapide. Voici une fonction pour mesurer le temps d’exécution d’une fonction :

In [ ]:
def mesurer_temps(func, *args, n_iterations=10, **kwargs):
    """
    Mesure le temps d'exécution moyen d'une fonction.
    
    Retourne
    --------
    tuple : (résultat, temps_moyen_ms)
    """
    times = []
    for _ in range(n_iterations):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        times.append(end - start)
    
    return result, np.mean(times) * 1000  # en millisecondes

Cette fonction est utilisable dans un script. Dans un notebook vous pouvez aussi mesurer le temps d’exécution d’une cellule avec les balises `%%time` sur la première ligne de la cellule. Attention, cela mesure le temps d’exécution de la cellule entière.

### 1.3 Génération d’images standardisées (mires)

Nous allons tester et comparer des fonctions pour le seuillage, pour un filtrage par convolution et pour de opérations morphologiques. Il est intéressant de créer des images contrôlées, qui contiennent des éléments (bruits, détails fins, fond, etc.) qui permettent d’éprouver ces opérations. 

Créons donc de telles images de manière procédurales.

Néanmoins nous vous demanderons aussi de traiter des images de votre choix afin de voir le rendu sur des éléments réels non standardisés aussi.

In [ ]:
def creer_image_test_seuillage():
    """Crée une image avec un histogramme bimodal pour tester le seuillage."""
    np.random.seed(42)
    h, w = 300, 400
    img = np.random.normal(180, 20, (h, w))  # Fond clair
    
    # Formes sombres
    img[50:150, 50:150] = np.random.normal(60, 15, (100, 100))  # Carré
    for i in range(h):
        for j in range(w):
            if (i - 150)**2 + (j - 300)**2 < 60**2:  # Cercle
                img[i, j] = np.random.normal(50, 10)
    img[200:280, 100:350] = np.random.normal(70, 15, (80, 250))  # Rectangle
    
    return np.clip(img, 0, 255).astype(np.uint8)


def creer_image_test_filtrage():
    """Crée une image avec du bruit pour tester les filtres."""
    np.random.seed(42)
    h, w = 300, 400
    
    # Image de base avec formes géométriques
    img = np.ones((h, w), dtype=np.float64) * 200
    img[50:150, 50:150] = 50  # Carré
    img[100:250, 200:220] = 60  # Rectangle vertical
    img[180:200, 100:350] = 60  # Rectangle horizontal
    
    # Ajout de bruit gaussien
    bruit = np.random.normal(0, 25, (h, w))
    img = img + bruit
    
    # Ajout de bruit sel et poivre
    for _ in range(500):
        y, x = np.random.randint(0, h), np.random.randint(0, w)
        img[y, x] = 255 if np.random.random() > 0.5 else 0
    
    return np.clip(img, 0, 255).astype(np.uint8)


def creer_image_test_morphologie():
    """Crée une image binaire pour tester les opérations morphologiques."""
    np.random.seed(42)
    h, w = 200, 300
    img = np.zeros((h, w), dtype=np.uint8)
    
    # Formes principales
    img[30:90, 30:100] = 255  # Rectangle
    for i in range(h):
        for j in range(w):
            if (i - 60)**2 + (j - 180)**2 < 40**2:  # Cercle
                img[i, j] = 255
    img[120:180, 50:120] = 255  # Carré avec trou
    img[135:165, 65:105] = 0   # Trou
    img[130:170, 180:250] = 255  # Rectangle avec petits trous
    img[145:155, 200:210] = 0
    img[145:155, 220:230] = 0
    
    # Bruit (petits points)
    for _ in range(100):
        y, x = np.random.randint(0, h), np.random.randint(0, w)
        img[y, x] = 255
    
    return img


# Création des images de test
img_seuillage = creer_image_test_seuillage()
img_filtrage = creer_image_test_filtrage()
img_morpho = creer_image_test_morphologie()

afficher_images(
    [img_seuillage, img_filtrage, img_morpho],
    ['Image pour seuillage', 'Image pour filtrage', 'Image pour morphologie']
)

## 2. Présentation des bibliothèques

### 2.1 OpenCV

**OpenCV** (Open Source Computer Vision Library) est une bibliothèque développée initialement par Intel, aujourd'hui maintenue par la communauté open-source.

- Site : https://opencv.org/
- Présentation sur Wikipédia : https://fr.wikipedia.org/wiki/OpenCV
- Dépôt GitHub : https://github.com/opencv/opencv
- Documentation : https://docs.opencv.org/

**Points forts :**
- Très performant (code optimisé en C++)
- Large écosystème (vision par ordinateur, machine learning, GPU)
- Standard industriel

**Particularités :**
- Images en BGR par défaut (pas RGB), mais fourni les outils pour convertir d’un espace colorimétrique à l’autre
- Certaines fonctions utilisent des conventions de spécification des dimensions (largeur, hauteur), il faut donc faire attention de ne pas se mélanger les pinceaux quand on appelle des méthodes NumPy avec les dimensions (lignes, colonnes), en se rappelant que le nombre de lignes correspondant à la hauteur et le nombre de colonne à la largeur.
- OpenCV travaille principalement en `uint8` (choix intuitif pour des images), donc bien envoyer les bons types au fonctions (si on définit une valeur par `value / 255.0` pour normaliser, le résultat sera un `float64`). Pour ne rien arranger, ce type d’erreur de typage peut-être silencieux (retourne un résulatat incohérent plutôt qu’une erreur).

### 2.2 scikit-image

**scikit-image** est une bibliothèque Python pure, intégrée à l'écosystème SciPy.
- Site : https://scikit-image.org/
- Présentation sur Wikipedia : https://fr.wikipedia.org/wiki/Scikit-image
- Dépôt GitHub : https://github.com/scikit-image/scikit-image
- Documentation : https://scikit-image.org/docs/stable/

**Points forts :**
- API « pythonique » et intuitive
- Excellente documentation et exemples
- Intégration native avec NumPy, SciPy, matplotlib

**Particularités :**
- Images en RGB ou niveaux de gris (convention standard)
- Travaille souvent avec des images normalisées [0, 1]
- Le type par défaut est (donc) généralement `float64` mais `uint8` est aussi utilisé pour les valeur [0, 255] (vérifier)

### 2.3 Tableau comparatif

| Critère | OpenCV | scikit-image |
|---------|--------|---------------|
| Performance | ⭐⭐⭐ Très rapide | ⭐⭐ Correct |
| Facilité d'utilisation | ⭐⭐ Moyenne | ⭐⭐⭐ Excellente |
| Documentation | ⭐⭐ Variable | ⭐⭐⭐ Excellente |
| Intégration Python | ⭐⭐ Wrapper C++ | ⭐⭐⭐ Native |
| Format d'image | BGR, uint8 | RGB, float [0,1] ou uint8 |

## 3. Seuillage et binarisation

### 3.1 Rappel théorique

Le seuillage transforme une image en niveaux de gris en image binaire :

$$
I'(x,y) = \begin{cases} 255 & \text{si } I(x,y) > T \\ 0 & \text{sinon} \end{cases}
$$

Le seuil optimal peut être déterminé automatiquement par la méthode d'**Otsu** qui minimise la variance intra-classe.

### 3.2 Seuillage avec OpenCV

Créez une fonction `seuil_cv()` qui réalise un seuillage simple et un seuillage d’Otsu à l’aide de la méthode `cv2.threeshold()` (regardez quels arguments sont nécessaires dans la doc).

Testez votre fonction sur l’image de test créée spécifiquement pour le seuillage.
Appliquez votre fonction de seuillage sur une image de votre choix.
Comparez différentes valeurs du seuil.

In [ ]:
# VOTRE CODE :


### 3.3 Seuillage avec Scikit-Image

Inspectez les méthodes de Scikit-Image importées dans ce notebook. Sélectionnez et utilisez celles qui vous semblent utiles pour réaliser une fonction de seuillage similaire à la précédente et que vous testerez de la même manière :

In [ ]:
# VOTRE CODE !


Scikit-image dispose de plusieurs autres méthodes pour déterminer un seuil :

```python
from skimage.filters import threshold_mean, threshold_minimum, threshold_triangle

seuil_mean = threshold_mean(img_seuillage)
seuil_triangle = threshold_triangle(img_seuillage)
```

Cherchez dans la doc comment elles déterminent les seuil, puis comparez-les :

In [ ]:
# VOTRE CODE :


### 3.4 Seuillage adaptatif

Le seuillage adaptatif calcule un seuil local pour chaque pixel, utile quand l'éclairage n'est pas uniforme.

Le seuillage global (comme Otsu) applique un seuil unique à toute l'image, ce qui fonctionne bien lorsque l'éclairage est uniforme. Cependant, dans de nombreuses situations réelles (documents scannés, photos avec ombres, éclairage naturel), la luminosité varie d'une zone à l'autre. Un seuil global ne peut alors pas séparer correctement les objets du fond sur l'ensemble de l'image.
Le seuillage adaptatif résout ce problème en calculant un seuil local pour chaque pixel, basé sur les intensités de son voisinage. Pour un pixel situé en $(x, y)$, on calcule une statistique (moyenne ou moyenne pondérée gaussienne) sur une fenêtre centrée autour de ce pixel, puis on soustrait une constante $C$ :

$$
T(x,y) = \text{moyenne}_{voisinage}(x,y) - C$$

Le pixel est alors classé blanc si son intensité est supérieure à ce seuil local, noir sinon.

Paramètres clés de l’algorithme :

- Taille du voisinage (blockSize) : détermine l'échelle des variations prises en compte. Une fenêtre trop petite sera sensible au bruit, une fenêtre trop grande ne s'adaptera pas aux variations locales. En pratique, il s’agit d’une valeur impaire (pour qu’il y ait un pixel centré). La taille du bloc dépend de la taille des objets et de l'échelle des variations d'éclairage. Pour un document scanné en 300 dpi, des valeurs entre 11 et 51 fonctionnent généralement bien. Pour des images plus petites, commencer avec des valeurs entre 7 et 15.
- Constante C : permet d'ajuster la sensibilité. Une valeur positive rend le seuillage plus strict (moins de pixels blancs).
- Méthode pour évaluer l’intensité du voisinnage : moyenne simple (MEAN) ou moyenne gaussienne (GAUSSIAN), cette dernière donnant plus de poids aux pixels proches du centre.

Créez des fonctions qui réalisent un seuillage adaptatif (avec le calcul de la moyenne, et le calcul de la gaussienne) avec les méthodes `cv2.adaptiveThreshold()` d’OpenCV et la méthode `skimage.filters.threeshold_local()` de Scikit-Image.

Tester ces fonctions avec l’image `img_seuillage` puis avec une image (« réelle ») de votre choix.

In [ ]:
# OpenCV : seuillage adaptatif
def seuil_adapt_cv(img, gauss=False):
   
    # GOTRE CODE

img_adapt_cv_mean = seuil_adapt_cv(img_seuillage)
img_adapt_cv_gauss = seuil_adapt_cv(img_seuillage, gauss=True)


# scikit-image : seuillage adaptatif
from skimage.filters import threshold_local

def seuil_adapt_ski(img, gauss=False):
   
    # VOTRE CODE

img_adapt_ski_mean = seuil_adapt_ski(img_seuillage)
img_adapt_ski_gauss = seuil_adapt_ski(img_seuillage, gauss=True)

afficher_images(
    [img_seuillage, img_adapt_cv_mean, img_adapt_cv_gauss, img_adapt_ski_mean, img_adapt_ski_gauss],
    ['Originale', 'Adaptatif Mean (CV)', 'Adaptatif Gauss (CV)', 'Adaptatif Mean (skimage)', 'Adaptatif Gauss (skimage)']
)

### 3.5 Exercice : Comparaison des méthodes de seuillage

Pour comparer les différentes méthodes de seuillage sur une image à éclairage non uniforme. nous allons d’abord créer une image avec un éclairage non-uniforme : 

In [ ]:
# Création d'une image avec éclairage non uniforme
def creer_image_eclairage_variable():
    h, w = 300, 400
    # Gradient d'éclairage
    y_grad = np.linspace(0.5, 1.5, h).reshape(-1, 1)
    x_grad = np.linspace(0.7, 1.3, w).reshape(1, -1)
    eclairage = y_grad * x_grad
    
    # Image de base
    img = np.ones((h, w)) * 180
    img[50:120, 50:150] = 50   # Rectangle 1
    img[180:250, 100:200] = 50  # Rectangle 2
    img[80:180, 280:350] = 50   # Rectangle 3
    
    # Application de l'éclairage
    img = img * eclairage
    img = np.clip(img, 0, 255).astype(np.uint8)
    
    return img

img_eclairage = creer_image_eclairage_variable()
afficher_images([img_eclairage],['Image éclairage variable'])

Testez et comparez les différentes méthodes de seuillage sur cette image :

- seuil otsu
- seuil adaptatif moyen
- seuil adaptatif gaussien

In [ ]:
# VOTRE CODE :


Testez différentes valeurs pour le paramètre `block_size` (scikit-image) et `blockSize` (opencv). Que constatez-vous ? Comment l’expliquez vous ?

Il semble donc que le filtre adaptatif n’est pas adapté quand on a de grandes plages uniformes.

Afin de corriger cela, vous pouvez essayer de modifier le paramètre $C$ (en forçant un seuil plus bas, plus de pixels seront classés noirs). Une autre approche consiste à combiner des approches en appliquer un filtre adaptatif, et de « combler » les trous avec une fermeture morphologique. 

## 4. Convolution et filtrage

### 4.1 Rappel théorique

La convolution applique un noyau $K$ à l'image $I$ :

$$
G(x,y) = \sum_{i,j} K(i,j) \cdot I(x+i, y+j)
$$

Les filtres classiques incluent :
- **Lissage** : moyenneur, gaussien (réduction du bruit)
- **Détection de contours** : Sobel, Laplacien
- **Netteté** : sharpen (pas vraiment un filtre en particulier)

### 4.2 Filtres de lissage

OpenCV propose les méthodes suivantes : `cv2.blur()`, `cv2.GaussianBlur()`, `cv2.medianBlur()`, `cv2.bilateralFilter()` 
Lisez la doc correspondante, testez les avec l’image `img_filtrage` que nous avons créé, et une image de votre choix.

In [ ]:
#VOTRE CODE :
# === OpenCV ===

# Filtre moyenneur (box filter)


# Filtre gaussien


# Filtre médian (non-linéaire, très efficace contre le bruit sel/poivre)


# Filtre bilatéral (préserve les contours)


afficher_images(
    [img_filtrage, img_blur_cv, img_gauss_cv, img_median_cv, img_bilateral_cv],
    ['Originale (bruitée)', 'Moyenneur 5×5', 'Gaussien 5×5 σ=1.5', 'Médian 5×5', 'Bilateral'],
    figsize=(16, 4)
)

Scikit-image propose des méthodes qui réalisent les mêmes opération (lisez attentivement les imports). Pour le filtre median nous pouvons utiliser un élément structurant (on vous donne le code). Pour les autres filtes, lisez la doc correspondante et testez-les de la même manière que précédemment

In [ ]:
# VOTRE CODE :
# === scikit-image ===

from skimage.filters import gaussian, median
from skimage.restoration import denoise_bilateral
from skimage.morphology import disk

# Conversion en float [0, 1] pour scikit-image


# Filtre gaussien

# Filtre médian (utilise un élément structurant)


# Filtre bilatéral

afficher_images(
    [img_filtrage, (img_gauss_ski * 255).astype(np.uint8), 
     img_median_ski, (img_bilateral_ski * 255).astype(np.uint8)],
    ['Originale', 'Gaussien (skimage)', 'Médian (skimage)', 'Bilatéral (skimage)'],
    figsize=(16, 4)
)

Qu’observez-vous lorsque l’on fait intervenir un élément structurant pour le filtre median (comparativement à OpenCV) ?

### 4.3 Détection de contours

Vous commencez à prendre le pli : testons les méthodes qui permettent d’extraire les contours de l’image `img_filtrage`.

Cette image est bruitée : nettoyons là un peu pour atténuer le bruit, qui pourrait géner la détection.m

In [ ]:
# Image nettoyée pour la détection de contours
img_clean = cv2.GaussianBlur(img_filtrage, (5, 5), 1)

Pour OpenCV les méthodes à tester devraient avoir des désignations familières : `cv2.Sobel()`, `cv2.Laplacian()`. Vous pouvez tester aussi la méthode `cv2.Canny()`. En fonction des résultat, tester différents traitements pour supprimerle bruit de `img_filtrage`, pour voir si cela a un impact.

In [ ]:
# VOTRE CODE :
# === OpenCV ===

# Sobel


# Laplacien


# Canny (détecteur de contours complet)


afficher_images(
    [img_clean, sobel_mag_cv, np.abs(laplacien_cv), canny_cv],
    ['Image lissée', 'Sobel (magnitude)', 'Laplacien', 'Canny'],
    figsize=(16, 4)
)

Comme précédemment, en ce qui concerne scikit-image nous vous importons les méthodes à tester :

In [ ]:
# VOTRE CODE :
# === scikit-image ===

from skimage.filters import sobel, sobel_h, sobel_v, laplace
from skimage.feature import canny



# Sobel


# Laplacien


# Canny


afficher_images(
    [img_clean, sobel_ski, np.abs(laplace_ski), canny_ski.astype(np.uint8) * 255],
    ['Image lissée', 'Sobel (skimage)', 'Laplacien (skimage)', 'Canny (skimage)'],
    figsize=(16, 4)
)

### 4.4 Convolution avec noyau personnalisé

OpenCV et Scikit-image vous donnent la possibilité de définir vos noyaux.

Définissez des noyaux de votre choix, par exemple deux noyaux 3×3, un de netteté (sharpen) et un de type relief (emboss). Vous pouvez bien sûr en définir plus, de toutes taille, etc. Expérimentez !

In [ ]:
# VOTRE CODE :
# Définition de noyaux personnalisés

# Noyau de netteté (sharpen)


# Noyau de relief (emboss)


Pour utiliser ces noyaux avec OpenCV, utilisez la méthode `cv2.filter2D` (ici aussi utilisez plutôt une version débruitée ou lissée de `img_filtrage`) :

In [ ]:
# VOTRE CODE :
# === OpenCV ===



afficher_images(
    [img_clean, img_sharpen_cv, img_emboss_cv],
    ['Originale', 'Sharpen', 'Emboss'],
    figsize=(15, 4)
)

Pour scikit-image :

In [ ]:
# === scikit-image ===
from scipy.ndimage import convolve


# pour « re-normaliser » l’image :



# pour « re-normaliser » l’image :


In [ ]:
afficher_images(
    [img_clean, img_sharpen_ski, img_emboss_ski],
    ['Originale', 'Sharpen', 'Emboss'],
    figsize=(15, 4)
)

Quelle différence constatez-vous entre les deux bibliothèques ?

### 4.5 Exercice : Pipeline de débruitage

Trouver la meilleure combinaison de filtres pour nettoyer l'image bruitée.

In [ ]:
# TODO : Expérimentez différentes combinaisons
# 1. Médian seul
# 2. Gaussien seul
# 3. Médian puis Gaussien
# 4. Bilatéral
#
# Conseil : Le filtre médian est très efficace contre le bruit sel/poivre
#           Le filtre gaussien lisse le bruit gaussien

# Votre code ici :
# ...

## 5. Morphologie mathématique

### 5.1 Rappel théorique

Les opérations morphologiques travaillent sur les **formes** des objets :

- **Érosion** : rétrécit les objets, supprime les petits détails
- **Dilatation** : agrandit les objets, comble les petits trous
- **Ouverture** (érosion → dilatation) : supprime les petits objets
- **Fermeture** (dilatation → érosion) : comble les petits trous

### 5.2 Éléments structurants

Voici comment crêéer des éléments structurants avec OpenCV et Scikit-image :

In [ ]:
# === OpenCV ===
# Création d'éléments structurants avec cv2.getStructuringElement

es_rect_cv = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
es_ellipse_cv = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
es_cross_cv = cv2.getStructuringElement(cv2.MORPH_CROSS, (5, 5))

print("OpenCV - Rectangle 5×5 :")
print(es_rect_cv)
print("\nOpenCV - Ellipse 5×5 :")
print(es_ellipse_cv)
print("\nOpenCV - Croix 5×5 :")
print(es_cross_cv)

In [ ]:
# === scikit-image ===
# Création d'éléments structurants avec skimage.morphology

from skimage.morphology import square, disk, diamond, rectangle

es_square_ski = square(5)
es_disk_ski = disk(2)  # rayon = 2 donne environ 5×5
es_diamond_ski = diamond(2)

print("scikit-image - Carré 5×5 :")
print(es_square_ski)
print("\nscikit-image - Disque (rayon 2) :")
print(es_disk_ski)
print("\nscikit-image - Diamant (rayon 2) :")
print(es_diamond_ski)

### 5.3 Opérations de base

- Créez un élément structurant : carré de 5 pixels de côté
- Appliquez les opérations suivantes :
    - érosion -> `cv2.erode()`
    - dilatation -> `cv2.dilate()`
    - ouverture -> `cv2.morphologyEx()` avec l’argument `cv2.MORPH_OPEN`
    - fermeture -> `cv2.morphologyEx()` avec l’argument `cv2.MORPH_CLOSE`

In [ ]:
# VOTRE CODE :
# === OpenCV ===

es =

img_erosion_cv = 
img_dilatation_cv = 
img_ouverture_cv = 
img_fermeture_cv =

afficher_images(
    [img_morpho, img_erosion_cv, img_dilatation_cv, img_ouverture_cv, img_fermeture_cv],
    ['Originale', 'Érosion', 'Dilatation', 'Ouverture', 'Fermeture'],
    figsize=(20, 4)
)

Avec Scikit-image vous pouvez importer directement les opérations `erosion`, `dilatation`, `opening` et `closing` (ainsique l’élément structurant `square`) depuis `skimage.morphology` : 

In [ ]:
# VOTRE CODE :
# === scikit-image ===

from skimage.morphology import erosion, dilation, opening, closing, square

es_ski = 

# scikit-image attend une image binaire (bool ou 0/1)
img_morpho_bool =

img_erosion_ski =
img_dilatation_ski = 
img_ouverture_ski = 
img_fermeture_ski = 

afficher_images(
    [img_morpho, img_erosion_ski.astype(np.uint8) * 255, 
     img_dilatation_ski.astype(np.uint8) * 255,
     img_ouverture_ski.astype(np.uint8) * 255, 
     img_fermeture_ski.astype(np.uint8) * 255],
    ['Originale', 'Érosion', 'Dilatation', 'Ouverture', 'Fermeture'],
    figsize=(20, 4)
)

### 5.4 Opérations avancées

Maintenant que vous disposez des opérations de base, vous pouvez créer des opérations plus avancées comme le gradient morphologique, top-hat, black-hat…

À ceci près que OpenCV propose également ces opérations : `cv2.MORPH_GRADIENT`, `cv2.MORPH_TOPHAT`, `cv2.MORPH_BLACKHAT`, etc.

Avec Scikit-image, il faut implémenter ces opérations (soustraction d’images, etc., cf. cours)

In [ ]:
# VOTRE CODE :
# === Gradient morphologique ===
# Gradient = Dilatation - Érosion (détecte les contours)

# OpenCV
gradient_cv =

# scikit-image
gradient_ski = 

# === Top-hat et Black-hat ===
# Top-hat = Original - Ouverture (extrait les petits éléments clairs)
# Black-hat = Fermeture - Original (extrait les petits éléments sombres)

tophat_cv = 
blackhat_cv = 

afficher_images(
    [img_morpho, gradient_cv, tophat_cv, blackhat_cv],
    ['Originale', 'Gradient morphologique', 'Top-hat', 'Black-hat'],
    figsize=(16, 4)
)

### 5.5 Morphologie en niveaux de gris

Pour le cours, afin de faciliter la compréhension, nous avons surtout présenté l’approche morphologique avec des images binaires. Nous avions indiqué que pour les images en niveau de gris, plutôt que prendre les valeurs 0 (érosion) et 1 (dilation), on prenait respectivement les valeurs min et max des pixels de l’ES. 

Cette opération est transparente pour l’utilisateur, par exemple définissez un élément structurant (par exemple une ellipse 7×7 – soit un cercle) et appliquez érosion, dilation et gradient sur l’image `img_seuillage`(qui est en niveau de gris) :

In [ ]:
# VOTRE CODE :
# La morphologie fonctionne aussi sur les images en niveaux de gris
# Érosion = minimum local, Dilatation = maximum local

es_gris = 

# OpenCV
img_erosion_gris_cv =
img_dilatation_gris_cv = 
img_gradient_gris_cv = 

afficher_images(
    [img_seuillage, img_erosion_gris_cv, img_dilatation_gris_cv, img_gradient_gris_cv],
    ['Originale', 'Érosion (min local)', 'Dilatation (max local)', 'Gradient'],
    figsize=(16, 4)
)

### 5.6 Exercice : Nettoyage d'un document scanné

Simulons un document scanné bruité, et comme dans le cours définissez un pipeline avec des opérations morphologiques pour nettoyer le document bruité :

In [ ]:
def creer_document_bruite():
    """Crée une image simulant un document scanné avec du bruit."""
    np.random.seed(42)
    h, w = 200, 300
    img = np.ones((h, w), dtype=np.uint8) * 240  # Fond clair
    
    # Simule des "lettres" (rectangles noirs)
    lettres = [
        (30, 30, 60, 15),   # y, x, hauteur, largeur
        (30, 55, 60, 15),
        (30, 80, 60, 15),
        (30, 105, 60, 15),
        (110, 30, 50, 12),
        (110, 50, 50, 12),
        (110, 70, 50, 12),
        (110, 90, 50, 12),
        (110, 110, 50, 12),
    ]
    for (y, x, h_l, w_l) in lettres:
        img[y:y+h_l, x:x+w_l] = 30
    
    # Grand rectangle (logo)
    img[30:100, 180:280] = 30
    img[45:85, 195:265] = 240  # Trou au milieu
    
    # Ajout de bruit (points isolés)
    for _ in range(300):
        y, x = np.random.randint(0, h), np.random.randint(0, w)
        img[y, x] = np.random.choice([0, 30, 240, 255])
    
    return img

img_document = creer_document_bruite()

# TODO : Nettoyez ce document
# 1. Binariser l'image (seuillage)
# 2. Supprimer le bruit avec une ouverture
# 3. Combler les petits trous avec une fermeture
# 4. Comparer le résultat avec et sans traitement morphologique

# Votre code ici :
# ...

afficher_images(
    [img_document, img_clean],
    ['Originale', 'Nettoyée (morphologie)'],
    figsize=(16, 4)
)

## 6. Comparaison des performances

Comparons les temps d'exécution entre OpenCV et scikit-image.

Complétez le code ci-dessous qui permet de comparer la réalisation des opérations morphologiques indiquées (en commentaire) par les deux bibliothèques. 

In [ ]:
# Création d'une grande image pour les tests de performance
img_grande = np.random.randint(0, 256, (1000, 1000), dtype=np.uint8)

# Test : Flou gaussien
_, t_gauss_cv = mesurer_temps(
_, t_gauss_ski = mesurer_temps(

# Test : Érosion
es_cv = cv2.getStructuringElement(cv2.MORPH_RECT, (7, 7))
es_ski = square(7)
_, t_erosion_cv = 
_, t_erosion_ski =

# Test : Sobel
_, t_sobel_cv = 
_, t_sobel_ski = 

# Affichage des résultats
print("Comparaison des performances (image 1000×1000)")
print("=" * 50)
print(f"{'Opération':<20} {'OpenCV (ms)':<15} {'scikit-image (ms)':<15} {'Ratio'}")
print("-" * 50)
print(f"{'Gaussien 15×15':<20} {t_gauss_cv:<15.2f} {t_gauss_ski:<15.2f} {t_gauss_ski/t_gauss_cv:.1f}x")
print(f"{'Érosion 7×7':<20} {t_erosion_cv:<15.2f} {t_erosion_ski:<15.2f} {t_erosion_ski/t_erosion_cv:.1f}x")
print(f"{'Sobel':<20} {t_sobel_cv:<15.2f} {t_sobel_ski:<15.2f} {t_sobel_ski/t_sobel_cv:.1f}x")

Qu’en concluez-vous ?

## 7. Exercice récapitulatif : Pipeline complet

**Objectif** : Créer un pipeline complet de traitement d'image pour détecter et compter des objets.

**Étapes** :
1. Charger/créer une image avec plusieurs objets
2. Prétraitement : conversion en niveaux de gris, filtrage du bruit
3. Segmentation : seuillage (Otsu ou adaptatif)
4. Nettoyage morphologique : ouverture/fermeture
5. Détection de contours
6. Comptage des objets

In [ ]:
def creer_image_objets():
    """Crée une image avec plusieurs objets à compter."""
    np.random.seed(123)
    h, w = 400, 500
    img = np.ones((h, w), dtype=np.uint8) * 200  # Fond gris clair
    
    # Ajout de plusieurs cercles (objets à compter)
    centres = [
        (80, 80), (80, 200), (80, 350), (80, 450),
        (200, 120), (200, 280), (200, 420),
        (320, 80), (320, 200), (320, 350), (320, 450),
    ]
    
    for (cy, cx) in centres:
        rayon = np.random.randint(25, 40)
        for i in range(h):
            for j in range(w):
                if (i - cy)**2 + (j - cx)**2 < rayon**2:
                    img[i, j] = np.random.randint(40, 80)
    
    # Ajout de bruit
    bruit = np.random.normal(0, 15, (h, w))
    img = np.clip(img + bruit, 0, 255).astype(np.uint8)
    
    return img, len(centres)

img_objets, n_objets_vrai = creer_image_objets()
print(f"Nombre réel d'objets : {n_objets_vrai}")

plt.figure(figsize=(10, 8))
plt.imshow(img_objets, cmap='gray')
plt.title(f'Image avec {n_objets_vrai} objets à détecter')
plt.axis('off')
plt.show()

In [ ]:
def creer_image_objets():
    """Crée une image avec plusieurs objets à compter, incluant des défauts."""
    np.random.seed(123)
    h, w = 400, 500
    img = np.ones((h, w), dtype=np.uint8) * 200  # Fond gris clair
    
    # Ajout de plusieurs cercles (objets à compter)
    centres = [
        (80, 80), (80, 200), (80, 350), (80, 450),
        (200, 120), (200, 280), (200, 420),
        (320, 80), (320, 200), (320, 350), (320, 450),
    ]
    
    for (cy, cx) in centres:
        rayon = np.random.randint(25, 40)
        for i in range(h):
            for j in range(w):
                if (i - cy)**2 + (j - cx)**2 < rayon**2:
                    img[i, j] = np.random.randint(40, 80)
    
    # ==========================================================================
    # DÉFAUTS : rendent le nettoyage morphologique nécessaire
    # ==========================================================================
    
    # 1. Trous/fissures dans les objets (pixels clairs à l'intérieur)
    for (cy, cx) in centres:
        # Petits trous aléatoires
        n_trous = np.random.randint(3, 7)
        for _ in range(n_trous):
            ty = cy + np.random.randint(-15, 15)
            tx = cx + np.random.randint(-15, 15)
            taille = np.random.randint(2, 5)
            img[ty-taille:ty+taille, tx-taille:tx+taille] = np.random.randint(180, 220)
        
        # Fissure horizontale ou verticale
        if np.random.random() > 0.5:
            # Fissure horizontale
            fy = cy + np.random.randint(-10, 10)
            img[fy:fy+2, cx-15:cx+15] = np.random.randint(190, 210)
        else:
            # Fissure verticale
            fx = cx + np.random.randint(-10, 10)
            img[cy-15:cy+15, fx:fx+2] = np.random.randint(190, 210)
    
    # 2. Bruit sel (petits points clairs sur les objets)
    for _ in range(100):
        py, px = np.random.randint(0, h), np.random.randint(0, w)
        img[py, px] = np.random.randint(200, 255)
    
    # 3. Bruit poivre (petits points sombres sur le fond = faux positifs potentiels)
    for _ in range(150):
        py, px = np.random.randint(0, h), np.random.randint(0, w)
        taille = np.random.randint(1, 4)
        img[max(0,py-taille):py+taille+1, max(0,px-taille):px+taille+1] = np.random.randint(40, 80)
    
    # 4. Petits artefacts isolés (petits objets parasites)
    for _ in range(20):
        ay, ax = np.random.randint(20, h-20), np.random.randint(20, w-20)
        rayon_artefact = np.random.randint(3, 8)
        for i in range(ay-rayon_artefact, ay+rayon_artefact):
            for j in range(ax-rayon_artefact, ax+rayon_artefact):
                if (i - ay)**2 + (j - ax)**2 < rayon_artefact**2:
                    if 0 <= i < h and 0 <= j < w:
                        img[i, j] = np.random.randint(40, 80)
    
    # Ajout de bruit gaussien global
    bruit = np.random.normal(0, 15, (h, w))
    img = np.clip(img + bruit, 0, 255).astype(np.uint8)
    
    return img, len(centres)

img_objets, n_objets_vrai = creer_image_objets()
print(f"Nombre réel d'objets : {n_objets_vrai}")

plt.figure(figsize=(10, 8))
plt.imshow(img_objets, cmap='gray')
plt.title(f'Image avec {n_objets_vrai} objets à détecter')
plt.axis('off')
plt.show()

In [ ]:
# TODO : Implémentez le pipeline complet
#
# Étape 1 : Filtrage du bruit (gaussien ou médian)
# img_filtree = ...
#
# Étape 2 : Seuillage (Otsu)
# img_binaire = ...
#
# Étape 3 : Nettoyage morphologique
# img_clean = ...
#
# Étape 4 : Détection de contours avec cv2.findContours
# contours, _ = cv2.findContours(img_clean, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# n_detectes = 
#
# Étape 5 : Affichage du résultat
img_result = cv2.cvtColor(img_objets, cv2.COLOR_GRAY2BGR)
cv2.drawContours(img_result, contours, -1, (0, 255, 0), 2)

# Visualisation du pipeline
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(img_objets, cmap='gray')
axes[0, 0].set_title('1. Image originale', fontweight='bold')
axes[0, 0].axis('off')

axes[0, 1].imshow(img_filtree, cmap='gray')
axes[0, 1].set_title('2. Filtrage médian', fontweight='bold')
axes[0, 1].axis('off')

axes[0, 2].imshow(img_binaire, cmap='gray')
axes[0, 2].set_title(f'3. Seuillage Otsu (T={seuil_otsu:.0f})', fontweight='bold')
axes[0, 2].axis('off')

axes[1, 0].imshow(img_clean, cmap='gray')
axes[1, 0].set_title('4. Nettoyage morphologique', fontweight='bold')
axes[1, 0].axis('off')

axes[1, 1].imshow(cv2.cvtColor(img_result, cv2.COLOR_BGR2RGB))
axes[1, 1].set_title(f'5. Résultat : {n_detecte} objets', fontweight='bold')
axes[1, 1].axis('off')

axes[1, 2].axis('off')
statut = "CORRECT" if n_detecte == n_objets_vrai else f"ERREUR ({n_objets_vrai} attendus)"
axes[1, 2].text(0.5, 0.5, f"Détectés : {n_detecte}\n\n{statut}", 
                transform=axes[1, 2].transAxes, fontsize=16, ha='center', va='center',
                fontweight='bold', color='green' if n_detecte == n_objets_vrai else 'red')

plt.suptitle('Pipeline : Détection et comptage d\'objets', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 8. Résumé et bonnes pratiques

### Quand utiliser OpenCV ?

- Applications temps réel (vidéo, caméra)
- Traitement de grandes images
- Besoin de performances maximales
- Écosystème vision par ordinateur (détection, tracking...)

### Quand utiliser scikit-image ?

- Prototypage rapide
- Analyse d'images scientifiques
- Intégration avec l'écosystème SciPy/NumPy
- Code plus lisible et maintenable

### Tableau de correspondance des fonctions principales

| Opération | OpenCV | scikit-image (à récup dans le bon module)|
|-----------|--------|---------------|
| Seuillage Otsu | `cv2.threshold(..., cv2.THRESH_OTSU)` | `threshold_otsu()` (filters)|
| Seuillage adaptatif | `cv2.adaptiveThreshold()` | `threshold_local()` (filters)|
| Flou gaussien | `cv2.GaussianBlur()` | `gaussian()` (filters)|
| Filtre médian | `cv2.medianBlur()` | `median()` (filters)|
| Sobel | `cv2.Sobel()` | `sobel()` (filters)|
| Canny | `cv2.Canny()` | `canny()` (filters)|
| Érosion | `cv2.erode()` | `erosion()` (morphology)|
| Dilatation | `cv2.dilate()` | `dilation()` (morphology)|
| Ouverture | `cv2.morphologyEx(..., MORPH_OPEN)` | `opening()` (morphology)|
| Fermeture | `cv2.morphologyEx(..., MORPH_CLOSE)` | `closing()` (morphology)|
| Élément structurant | `cv2.getStructuringElement()` | `square()`, `disk()`, `diamond()` (morphology) |

**Pour allez plus loin** : 

* [TP optionnel](Data-Science-05-TP_Detection_Visages_Haar_LFW.ipynb) sur la détection de visage avec OpenCV
* [Une collection de Notebooks](https://haesleinhuepf.github.io/BioImageAnalysisNotebooks/intro.html#) de Robert Haase qui montre de nombreuses techniques de traitement d’images biologiques/médicales, de l’affichage au machine learning. Vous pouvez notamment porter attention aux sections « image filtering » et « image segmentation ».